In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)   #to display all the column information
pd.options.display.max_seq_items = 2000

### Combine the separate log files fo each wells

In [2]:
def MergeCsv(file_directory,depth_col_name,*file):
    file_path = os.path.join(file_directory,file[0])
    data = pd.read_csv(file_path)
    
    for i in range(1,len(file)):
        file_path = os.path.join(file_directory,file[i])
        df = pd.read_csv(file_path)
        data = pd.merge(data, df, on = depth_col_name)
    return data

In [3]:
file_name1 = "Boggess_AnisoMechPro.csv"
file_name2 = "Boggess_PetroInterp.csv"
# file_directory = r"C:\Users\tmo0005\Desktop\Thesis work\Thesis work\Well_Data_CSV"
file_directory = r"../Thesis work/Thesis work/Well_Data_CSV" 
depth_col_name = "TDEP"

data_boggess =  MergeCsv(file_directory,depth_col_name, file_name1, file_name2)

In [4]:
file_name1 = "MIP_3H_IsoMechPro.csv"
file_name2 = "MIP_3H_PetroInterp.csv"
# file_directory = r"C:\Users\tmo0005\Desktop\Thesis work\Thesis work\Well_Data_CSV"
file_directory = r"../Thesis work/Thesis work/Well_Data_CSV" 
depth_col_name = "DEPT"

data_mip3h =  MergeCsv(file_directory,depth_col_name, file_name1, file_name2)

In [5]:
file_name1 = "Whipkey_Main.csv"
file_name2 = "Whipkey_Sonic.csv"
# file_directory = r"C:\Users\tmo0005\Desktop\Thesis work\Thesis work\Well_Data_CSV"
file_directory = r"../Thesis work/Thesis work/Well_Data_CSV" 
depth_col_name = "DEPT"

data_whipkey =  MergeCsv(file_directory,depth_col_name, file_name1, file_name2)

In [6]:
file_name1 = "Poseidon_AnisoMechPro.csv"
file_name2 = "Poseidon_PetroInterp.csv"
# file_directory = r"C:\Users\tmo0005\Desktop\Thesis work\Thesis work\Well_Data_CSV"
file_directory = r"../Thesis work/Thesis work/Well_Data_CSV" 
depth_col_name = "TDEP"

data_poseidon =  MergeCsv(file_directory,depth_col_name, file_name1, file_name2)

### Calculate the brittleness for each wells

In [8]:
YM_brit = (data_poseidon['YME_DYN']-data_poseidon['YME_DYN'].min())/(data_poseidon['YME_DYN'].max()-data_poseidon['YME_DYN'].min())
PR_brit = (data_poseidon['PR_DYN']-data_poseidon['PR_DYN'].min())/(data_poseidon['PR_DYN'].max()-data_poseidon['PR_DYN'].min())

df =  1/ data_poseidon['PR_DYN']
PR_brit_inv = (df - df.min())/(df.max() - df.min())

data_poseidon['Brittleness'] = (YM_brit + PR_brit) / 2
data_poseidon['Brittleness_new'] = (YM_brit + (PR_brit_inv)) / 2

In [9]:
YM_brit = (data_boggess['YME_DYN']-data_boggess['YME_DYN'].min())/(data_boggess['YME_DYN'].max()-data_boggess['YME_DYN'].min())
PR_brit = (data_boggess['PR_DYN']-data_boggess['PR_DYN'].min())/(data_boggess['PR_DYN'].max()-data_boggess['PR_DYN'].min())

df =  1/ data_boggess['PR_DYN']
PR_brit_inv = (df - df.min())/(df.max() - df.min())

data_boggess['Brittleness'] = (YM_brit + PR_brit) / 2
data_boggess['Brittleness_new'] = (YM_brit + (PR_brit_inv)) / 2

In [10]:
YM_brit = (data_mip3h['YME_DYN']-data_mip3h['YME_DYN'].min())/(data_mip3h['YME_DYN'].max()-data_mip3h['YME_DYN'].min())
PR_brit = (data_mip3h['PR']-data_mip3h['PR'].min())/(data_mip3h['PR'].max()-data_mip3h['PR'].min())

df =  1/ data_mip3h['PR']
PR_brit_inv = (df - df.min())/(df.max() - df.min())

data_mip3h['Brittleness'] = (YM_brit + PR_brit) / 2
data_mip3h['Brittleness_new'] = (YM_brit + (PR_brit_inv)) / 2

In [12]:
#calculate elastic property for whipkey
Vp = 116.08 / data_whipkey['DT']
Vs = 116.08 / data_whipkey['DTSM_FAST']
Rhob = data_whipkey['RHOB']

v = (Vp**2 - 2*Vs**2) / (2*(Vp**2 - Vs**2))
E = (Rhob * Vs**2 * (3*Vp**2 - 4*Vs**2)) / (Vp**2 - Vs**2)

In [13]:
YM_brit = (E-E.min())/(E.max()-E.min())
PR_brit = (v-v.min())/(v.max()-v.min())

df =  1/ v
PR_brit_inv = (df - df.min())/(df.max() - df.min())

data_whipkey['Brittleness'] = (YM_brit + PR_brit) / 2
data_whipkey['Brittleness_new'] = (YM_brit + (PR_brit_inv)) / 2

### change the column names to the same

In [16]:
data_poseidon.rename(columns = {"TDEP":"DEPT", "NPHI_x":"NPHI", "RHOZ_x":"RHOZ"}, inplace = True)

In [17]:
data_boggess.rename(columns = {"TDEP":"DEPT", "RHOZ_x":"RHOZ"}, inplace = True)

In [18]:
data_mip3h.rename(columns = {"GR_x":"GR"}, inplace = True)

In [19]:
data_whipkey.rename(columns = {"DT":"DTCO", "GR_x":"GR"}, inplace = True)

### saving the new data

In [20]:
###creates a folder to save the csv files
###checks if a folder called Well_Data_CSV already exists to avoid error
new_file_directory = os.path.join(os.path.split(file_directory)[0], "Well_Data_CSV_Merged")
if os.path.exists(new_file_directory):
    pass
else:
    os.mkdir(os.path.join(os.path.split(new_file_directory)[0],"Well_Data_CSV_Merged"))

In [21]:
data_poseidon.to_csv(os.path.join(new_file_directory,"Poseidon.csv"))

In [22]:
data_boggess.to_csv(os.path.join(new_file_directory,"Boggess.csv"))

In [23]:
data_mip3h.to_csv(os.path.join(new_file_directory,"Mip3h.csv"))

In [24]:
data_whipkey.to_csv(os.path.join(new_file_directory,"Whipkey.csv"))